In [ ]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports 
from classical_shadow_n_clifford import ClassicalShadow_N_CLIFFORD
from shadow_protocol import ShadowProtocol

from qiskit import QuantumCircuit
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Statevector
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_aer import AerSimulator
from qiskit.circuit import Parameter, ParameterVector
import qiskit.qasm3
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator, QiskitRuntimeService
from visualisation import LiveVisualizer
import matplotlib.pyplot as plt
import random

In [ ]:
# Setup Experiment
class Bell_Protocol(ShadowProtocol):

    def get_num_qubits(self) -> int:
        return 2
    
    def get_state_circuit(self) -> QuantumCircuit:
        circuit = QuantumCircuit(2)
        return circuit

    def run_circuit_and_get_measurement(self, circuit) -> list[int]:
        sim = AerSimulator()

        # Run with 997 shots
        job = sim.run(circuit, shots=997)
        result = job.result()

        counts = result.get_counts()
        max_hits= max(counts, key=counts.get)
        return [int(bit) for bit in list(max_hits)]
    
bell_protocol = Bell_Protocol()
# Create Classical Shadow Instance
bell_shadow = ClassicalShadow_N_CLIFFORD(bell_protocol)

In [ ]:
target_density_matrix = bell_shadow.get_original_density_matrix()
vis = LiveVisualizer(target_density_matrix)


while True:
    bell_shadow.add_snapshot()
    print(f"Shadow Size: {bell_shadow.get_shadow_size()}")
    

    actual_density_matrix = bell_shadow.get_density_matrix_from_stabilizers()
    
    vis.update(actual_density_matrix,bell_shadow.get_shadow_size()) 